# Final Presentation ACDC

In this notebook we will walk through examples of this package. The following points are covered: 
- Find downstream vertices
- Find alternative edges
- Handle PGM input format
- Raise errors if the data is incorrect. 
- Aggregate the power flow results in tables. 
- Show tables with data of powerflow calculations. 
- Calculate EV penetration levels. 
- Calculate optimal tap positions. 
- Do N-1 Calculations. 

In [ ]:
# some basic imports

import pytest
import graph_processing as tp  # Import power_system_simpulation.graphy_processing
import pandas as pd
import numpy as np
import scipy as sp

import power_flow_processing as pfp
from graph_processing import (
    EdgePairNotUniqueError,
    GraphCycleError,
    GraphNotFullyConnectedError,
    GraphProcessor,
    IDNotFoundError,
    IDNotUniqueError,
    InputLengthDoesNotMatchError,
)

# 1.1 Input dataset

We create an input graph by using the following parameters: 
- `vertex_ids`
- `edge_ids`
- `edge_vertex_id_pairs`
- `edge_enabled`
- `source_vertex_id`

In [ ]:
vertex_ids = [0, 2, 4, 6, 10]  # All unique vertex ids
edge_ids = [1, 3, 5, 7, 8, 9]  # All unique edge ids
edge_vertex_id_pairs = [  # Which vertex ids are connected by an edge
    (0, 2),  # edge 1
    (0, 4),  # edge 3
    (0, 6),  # edge 5
    (2, 4),  # edge 7
    (4, 6),  # edge 8
    (2, 10),  # edge 9
]
edge_enabled = [True, True, True, False, False, True]  # Whether each edge is enabled or disabled
source_vertex_id = 0  # ID of the source vertex

This graph will result in this visual representation: 

In [ ]:
"""      
vertex_0 (source) --edge_1(enabled)-- vertex_2 --edge_9(enabled)-- vertex_10
                 |                               |
                 |                           edge_7(disabled)
                 |                               |
                 -----------edge_3(enabled)-- vertex_4
                 |                               |
                 |                           edge_8(disabled)
                 |                               |
                 -----------edge_5(enabled)-- vertex_6
"""

# 1.2 Validation
This graph is tested for the following conditions: 
1. `vertex_ids` and `edge_ids` should be unique.
    - This function compares the length of all `vertex_ids` to the set of `vertex_ids` and gives an error if they are not the same. It uses the same approach for `edge_ids`.
2. `edge_vertex_id_pairs` should have the same length as `edge_ids`.
    - This function compares the length of the list of `edge_vertex_id_pairs` and `edge_ids`.
3. `edge_vertex_id_pairs` should contain valid `vertex ids`.
    - Using a loop all `edge_vertex_id_pairs` are checked to also be valid `edge_ids`.
4. `edge_enabled` should have the same length as `edge_ids`.
    - The length of the `edge_enabled` list is compared to length of the `edge_ids` list. 
5. `source_vertex_id` should be a valid `vertex_ids`.
     - The `source_vertex_id` is checked to be part of `vertex_ids`.
6. The graph should not contain cycles.
    - An adjacency list is built and using depth first search cycles are detected. 
7. The graph should be fully connected.
    - Using depth fist search the length of all the visited vertex and the length of `vertex_ids` is compared. 
8. Multiple edges should not connect the same two `vertex_ids`. 
    - A list of `edge_vertex_id_pairs` is compared to a list of the set of `edge_vertex_id_pairs`.

This can be tested by calling the `tp.GraphProcessor` function: 

In [ ]:
test2 = tp.GraphProcessor(
    vertex_ids=vertex_ids,
    edge_ids=edge_ids,
    edge_vertex_id_pairs=edge_vertex_id_pairs,
    edge_enabled=edge_enabled,
    source_vertex_id=source_vertex_id,
)

# Example
When for example not all `vertex_ids` are unique the following error will be raised: 

In [ ]:
with pytest.raises(IDNotUniqueError) as excinfo:
    GraphProcessor([1, 2, 3, 3, 5], [1, 2, 3], [(1, 2), (2, 3), (1, 5)], [True, True, True], 1)
assert str(excinfo.value) == "Vertex IDs are not unique"

In [ ]:
test = tp.GraphProcessor(
    vertex_ids=vertex_ids,
    edge_ids=edge_ids,
    edge_vertex_id_pairs=edge_vertex_id_pairs,
    edge_enabled=edge_enabled,
    source_vertex_id=source_vertex_id,
)

# 1.3 Find downstream vertices

 Given an `edge_id`, return all the vertices which are in the downstream of the edge, with respect to the source vertex. Including the downstream vertex of the edge itself!
 Only the `edge_enabled` are taken into account. 
 The function returns a list of all downstream `vertex_ids`. 

In [ ]:
vertex_ids = [0, 2, 4]  # All unique vertex ids
edge_ids = [1, 3]  # All unique edge ids
edge_vertex_id_pairs = [(0, 2), (2, 4)]  # Egde 1 and egde 3
edge_enabled = [True, True]  # Whether each edge is enabled or disabled
source_vertex_id = 0  # ID of the source vertex

test3 = tp.GraphProcessor(
    vertex_ids=vertex_ids,
    edge_ids=edge_ids,
    edge_vertex_id_pairs=edge_vertex_id_pairs,
    edge_enabled=edge_enabled,
    source_vertex_id=source_vertex_id,
)

downstream_vertices = test3.find_downstream_vertices(1)
print(downstream_vertices)

# 1.4 Find alternative edges 
Given an enabled edge the following analysis is done: 
- If this edge is going to be disabled. 
- Which (currently disabled) edge can be enabled to ensure that the graph is again fully connected and acyclic?
- Return a list of all alternative edges.

Our example graph would return the following results:   
        Call find_alternative_edges with disabled_edge_id=1 will return [7]   
        Call find_alternative_edges with disabled_edge_id=3 will return [7, 8]   
        Call find_alternative_edges with disabled_edge_id=5 will return [8]   
        Call find_alternative_edges with disabled_edge_id=9 will return []   

This function can be used by using the `find_alternative_edges` function and giving an `edge_ids` as input. 

In [ ]:
alternative_edges = test2.find_alternative_edges(1)
print("Alternative edge when disabling edge 1 is:", alternative_edges)

alternative_edges = test2.find_alternative_edges(3)
print("Alternative edge when disabling edge 3 are:", alternative_edges)

alternative_edges = test2.find_alternative_edges(5)
print("Alternative edge when disabling edge 5 are:", alternative_edges)

alternative_edges = test2.find_alternative_edges(9)
print("Alternative edge when disabling edge 9 are:", alternative_edges)

# 2.1 Power Grid Model (PGM) Input

The following sections describe a power grid calculation module with the use of the `power-grid-model` library. Firstly, we must handle the following inputs:
- A power grid in PGM format
- A table containing active load profile of all the `sym_load` in the grid, with timestamps and load ids.
- A table containing ractive load profile of all the `sym_load` in the grid, with timestamps and load ids.

This data can be imported using the following code:

In [ ]:
from power_grid_model.utils import json_deserialize_from_file

grid_data = json_deserialize_from_file("input_network_data.json")
active_power_profile = pd.read_parquet("active_power_profile.parquet")
reactive_power_profile = pd.read_parquet("reactive_power_profile.parquet")

The `grid_data` consists of several different tables containing information about different elements of the grid, like lines, nodes, sources and load information.

In [ ]:
print(pd.DataFrame(grid_data["line"]))
print(pd.DataFrame(grid_data["node"]))
print(pd.DataFrame(grid_data["source"]))
print(pd.DataFrame(grid_data["sym_load"]))

The `active_power_profile` and `reactive_power_profile` contain the power profiles for different loads, so they tell how much power should be supplied to for example a household or factory (loads) at what time.

In [ ]:
print(active_power_profile)
display(reactive_power_profile)

# 2.2 Constructing PGM using input data

Using the previously imported input data, a power grid model (PGM) can be constructed. Furthermore, validation is important in this step, and as such an error should be raised if the data is invalid. The PGM can be constructed by calling the class `pfp.PowerFlow`. 

In [ ]:
pgm = pfp.PowerFlow(grid_data)

The `PowerFlow` class contains an initialization function where the PGM is constructed, and the data is validated using the following line of code, which is already present in the actual function. 

In [ ]:
pfp.assert_valid_input_data(input_data=grid_data, symmetric=True, calculation_type=pfp.CalculationType.power_flow)

# 2.3 PGM Batch update dataset and power flow calculation

The batch update dataset includes multiple power profiles (active and reactive power) for various nodes in the grid over a specified period. Instead of updating the grid model for each individual timestamp or node separately, the entire set of updates is applied at once, allowing us to perform the power flow calculation for the entire period.

Within the `batch_powerflow` function, the power flow calculation is also performed. This is done using the Newton-Raphson method, which returns the `output_data` which contains the solution to the power flow calculation. Furthermore, the batch update dataset is also validated within the function.

This entire functionality can easily be performed with the following line of code:

In [ ]:
output_data = pgm.batch_powerflow(active_power_profile, reactive_power_profile)

# 2.4 Aggregating Power Flow Results

Now all the data has been prepared and the state of the power grid after applying all batch updates has been computed, the power flow results will be aggregated in two tables:
- A voltage table with each row representing a timestamp and containing the following columns:
    - Timestamp (index column)
    - Maximum p.u. voltage of all the nodes for this timestamp
    - The node ID with the maximum p.u. voltage
    - Minimum p.u. voltage of all the nodes for this timestamp
    - The node ID with the minimum p.u. voltage

- A loading table with each row representing a line and containing the following columns:
    - Line ID (index column)
    - Total energy loss of the line in kWh over the entire period
    - Maximum loading in p.u. of the line across the whole timeline
    - Timestamp of this maximum loading moment
    - Minimum loading in p.u. of the line across the whole timeline
    - Timestamp of this minimum loading moment

These tables are constructed using two separate functions, which also gives the results for the given input data and power profiles:

In [ ]:
pgm.aggregate_voltage_table(active_power_profile, reactive_power_profile)

In [ ]:
pgm.aggregate_loading_table(active_power_profile, reactive_power_profile)

# 3.1 Using a LV grid with a MV/LV transformer

This part of the package will present low voltage grid analytics functions. These analytics include EV penetration level and optimal tap position. Input data can be given as: 
- A LV grid in PGM input format.
- LV feeder IDs list.
- A (active and reactive) load profile.
- A pool of EV charging profiles for the same time period as the time period of load profile.

The data is validated to check for the following criteria: 
- The LV grid should be a valid PGM input data.
- The LV grid has exactly one `transformer`, and one `source`.
- All IDs in the LV Feeder IDs are valid line IDs.
- All the lines in the LV Feeder IDs have the `from_node` the same as the `to_node` of the `transformer`.
- The grid is fully connected in the initial state.
- The grid has no cycles in the initial state.
- The timestamps are matching between the active load profile, reactive load profile, and EV charging profile.
- The IDs in active load profile and reactive load profile are matching.
- The IDs in active load profile and reactive load profile are valid IDs of `sym_load`.
- The number of EV charging profile is at least the same as the number of `sym_load`.

In [ ]:
import os
import math
import pandas as pd
import graph_processing as tp
from power_grid_model.utils import json_deserialize_from_file
import power_flow_processing as pfp
import power_system_simulation as pss

number_of_houses = 4
number_of_feeders = 2
penetration_level = 0.80
# total_evs = number_of_houses * penetration_level
# evs_per_feeder = math.floor(total_evs / number_of_feeders)
# print(f"EVs per feeder: {evs_per_feeder}")

# Determine the file path relative to this script's location
grid_data = json_deserialize_from_file("input_network_data_assign3.json")
active_power_profile = pd.read_parquet("active_power_profile.parquet")
ev_active_power_profile = pd.read_parquet("ev_active_power_profile.parquet")
reactive_power_profile = pd.read_parquet("reactive_power_profile.parquet")

EV_test = pss.PowerSim(grid_data=grid_data)

# print(ev_active_power_profile.columns)


# Randomly select houses for EV assignment within each feeder
# These values should be given as an input from another file
# Within a LV feeder, randomly select houses which will have EVs
# For each selected house with EV, randomly select an EV charging profile to add to the sym_load of that house.
# After assignment of EV profiles, run a time-series power flow as in Assignment 2, return the two aggregation tables.

# edge_vertex_id_pairs = list(zip(grid_data["line"]["from_node"], grid_data["line"]["to_node"])) + list(
#             zip(grid_data["transformer"]["from_node"], grid_data["transformer"]["to_node"])
#         )
# edge_enabled = []
# for i in grid_data["line"]["id"]:
#             index = np.where(grid_data["line"]["id"] == i)
#             if grid_data["line"][index]["from_status"] == 1 & grid_data["line"][index]["to_status"] == 1:
#                 edge_enabled = edge_enabled + [True]
#             else:
#                 edge_enabled = edge_enabled + [False]
# if grid_data["transformer"][0]["from_status"] == 1 & grid_data["transformer"][0]["to_status"] == 1:
#             edge_enabled = edge_enabled + [True]
# else:
#             edge_enabled = edge_enabled + [False]
# source_vertex_id = grid_data["source"]["node"][0]
# edge_ids = list(grid_data["line"]["id"]) + list(grid_data["transformer"]["id"])
# vertex_ids = grid_data["node"]["id"]
# test4 = tp.GraphProcessor(
#         vertex_ids=vertex_ids,
#         edge_ids=edge_ids,
#         edge_vertex_id_pairs=edge_vertex_id_pairs,
#         edge_enabled=edge_enabled,
#         source_vertex_id=source_vertex_id,
#     )
# transformer_to_node = grid_data["transformer"][0]["to_node"]

# # assigned_profiles = set()   # This keeps track of the assigned profiles

# for line in grid_data["line"]:
#     if line["from_node"] == transformer_to_node:
#         print(f"Found line with from_node {transformer_to_node}: to_node = {line['to_node']}")
#         print(f"Edge details: id = {line['id']}, r1 = {line['r1']}, x1 = {line['x1']}, ...")  # Find all feeders
#         downstream_vertices = test4.find_downstream_vertices(line['id'])
#         #print(downstream_vertices)
#         sym_load_nodes = set(grid_data['sym_load']['node'])
#         common_nodes = sym_load_nodes.intersection(downstream_vertices)
#         print(common_nodes) # all the nodes in a downstream vertex that have a sym load
#         '''
#         # ---- asigment of profiles
#         ev_assignment_counter = 0
#         for node in common_nodes:
#             if ev_assignment_counter >= evs_per_feeder:
#                 break
#     for idx, row in ev_active_power_profile.iterrows():
#                 if idx not in assigned_profiles:
#                     # Assign EV profile to active_power_profile at the corresponding node
#                     active_power_profile.loc[active_power_profile['node'] == node, 'ev_profile'] = idx
#                     assigned_profiles.add(idx)
#                     ev_assignment_counter += 1
#                     break
#                 if ev_assignment_counter >= evs_per_feeder:
#                     break  # Stop further processing of lines once we've assigned enough EV profiles
#         '''